In [ ]:
#Reading the traffic excel file data in Pandas
import pandas as pd

read_file = pd.read_csv(r"g:\Guvi-Datascience\Project\Secure Check_Project 1\traffic_stops - traffic_stops_with_vehicle_number.csv")

In [ ]:

read_file.isnull().sum()

In [ ]:
#Filling the NAN Values
read_file.fillna(
    {
        "search_type" : 'Unknown'
    }, inplace = True

)

In [ ]:
read_file.info()

In [ ]:
#converting object data type of date and time into date datatype
read_file['stop_date'] = pd.to_datetime(read_file['stop_date'], format='%d-%m-%Y', errors='coerce')
read_file['stop_time'] = pd.to_datetime(read_file['stop_time'], format='%H:%M:%S', errors='coerce').dt.time


In [ ]:
#concatinate Date and Time as Timestamp
read_file['stop_timestamp'] = pd.to_datetime(read_file['stop_date'].astype(str) + ' ' + read_file['stop_time'].astype(str),errors='coerce')

In [ ]:
#Mysql Connection
import mysql.connector

mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    password="Pugazh@08",
    database="police_db"
)

In [ ]:
mycursor=mydb.cursor(dictionary=True)

In [ ]:
mycursor.execute("Create database police_db;")


In [ ]:
mycursor.execute("Use police_db;")

In [ ]:
#Creating police_logs table on the Database police_db

mycursor.execute("""
CREATE TABLE IF NOT EXISTS police_logs (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(18),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    drugs_related_stop BOOLEAN,
    stop_duration VARCHAR(50),
    vehicle_number VARCHAR(50),
    stop_timestamp DATETIME
)
""")

In [ ]:
#inserting the Values into the table

insert_query = """
INSERT INTO police_logs (
    stop_date, stop_time, country_name, driver_gender, driver_age, driver_race,
    violation, search_conducted, search_type, stop_outcome, drugs_related_stop,
    stop_duration, vehicle_number, stop_timestamp
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for index, row in read_file.iterrows():
    mycursor.execute(insert_query, (
        row['stop_date'],
        row['stop_time'],
        row['country_name'],
        row['driver_gender'],
        row['driver_age'],
        row['driver_race'],
        row['violation'],
        row['search_conducted'],
        row['search_type'],
        row['stop_outcome'],
        row['drugs_related_stop'],
        row['stop_duration'],
        row['vehicle_number'],
        row['stop_timestamp']
    ))


mydb.commit()
mydb.close()


In [ ]:

mycursor = mydb.cursor(dictionary=True)

mycursor.execute("SELECT * FROM police_logs;")
rows = mycursor.fetchall()
data = pd.DataFrame(rows)

csv_count = len(read_file)
database_data_count= len(data)

print(f"CSV record count: {csv_count}")
print(f"Database record count: {database_data_count}")

mycursor.close()